Data Ingestion

In [2]:
### document structure

from langchain_core.documents import Document

In [ ]:
doc = Document(
    page_content = "this is the main text content I am using to create RAG",
    metadata={
        "source":"example.txt",
        "pages":1,
        "author": "Mr Potato",
        "date_created":"2026-01-01"
    }
)
# doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Mr Potato', 'date_created': '2026-01-01'}, page_content='this is the main text content I am using to create RAG')

In [8]:
## create a simple txt file
import os
os.makedirs("../data/text_files", exist_ok=True)

In [6]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath, content in sample_texts.items():
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)

print("Sample text files created!")

Sample text files created!


In [13]:
### TextLoader
# from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")

document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [14]:
### Directory Loader

from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader = DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", ## Pattern to match files
    loader_cls = TextLoader, ##loader class to use
    loader_kwargs = {'encoding': 'utf-8'},
    show_progress = False
)

documents=dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popu

In [17]:

from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf", ## Pattern to match files  
    loader_cls= PyMuPDFLoader, ##loader class to use
    show_progress=False

)

pdf_documents=dir_loader.load()


In [20]:
# Creating Data Chunks 

from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """
    Split documents into smaller chunks for better RAG performance.
    
    Parameters:
    - chunk_size: Maximum characters per chunk (adjust based on your LLM)
    - chunk_overlap: Characters to overlap between chunks (preserves context)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, # Each chunk: ~1000 characters
        chunk_overlap=chunk_overlap, # 200 chars overlap for context
        length_function=len, # How to measure length
        separators=["\n\n", "\n", " ", ""] # Split hierarchy
    )
    # Actually split the documents
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show what a chunk looks like
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs
    

In [23]:
chunks = split_documents(pdf_documents)

Split 13 documents into 93 chunks

Example chunk:
Content: Received 10 February 2023, accepted 25 February 2023, date of publication 8 March 2023, date of current version 13 March 2023.
Digital Object Identifier 10.1109/ACCESS.2023.3251892
Explainable Misinfo...
Metadata: {'producer': 'pdfTeX-1.40.24; modified using iText® 7.1.1 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-03-10T10:10:14+05:30', 'source': '..\\data\\pdf\\Explainable_Misinformation_Detection_Across_Multiple_Social_Media_Platforms.pdf', 'file_path': '..\\data\\pdf\\Explainable_Misinformation_Detection_Across_Multiple_Social_Media_Platforms.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Explainable Misinformation Detection Across Multiple Social Media Platforms', 'author': '', 'subject': 'IEEE Access;2023;11; ;10.1109/ACCESS.2023.3251892', 'keywords': '', 'moddate': '2023-03-10T21:29:36-05:00', 'trapped': 'False', 'modDate': "D:20230310212936-05'00'", 'creatio

### Embedding and Vector DB

In [18]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args: 
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embeddings_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

## initialize the embedding manager
embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 421.19it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


VectorStore

In [31]:
from typing import List, Any
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str="../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            #Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            #Get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number")
        print(f"Adding {len(documents)} documents to vector store...")

        #Prepare for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            #Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            #Document content
            documents_text.append(doc.page_content)

            #Embedding
            embeddings_list.append(embedding.tolist())

        #Add to Collection
        try:
            self.collection.add(
                ids = ids,
                embeddings = embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [24]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.24; modified using iText® 7.1.1 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-03-10T10:10:14+05:30', 'source': '..\\data\\pdf\\Explainable_Misinformation_Detection_Across_Multiple_Social_Media_Platforms.pdf', 'file_path': '..\\data\\pdf\\Explainable_Misinformation_Detection_Across_Multiple_Social_Media_Platforms.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Explainable Misinformation Detection Across Multiple Social Media Platforms', 'author': '', 'subject': 'IEEE Access;2023;11; ;10.1109/ACCESS.2023.3251892', 'keywords': '', 'moddate': '2023-03-10T21:29:36-05:00', 'trapped': 'False', 'modDate': "D:20230310212936-05'00'", 'creationDate': "D:20230310101014+05'30'", 'page': 0}, page_content='Received 10 February 2023, accepted 25 February 2023, date of publication 8 March 2023, date of current version 13 March 2023.\nDigital Object Identifier 10.1109/ACCESS.2023.3251892\nExplainabl

In [32]:
### convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generate the Embeddings
embeddings = embedding_manager.generate_embeddings(texts)

## Store in the vector db
vectorstore.add_documents(chunks, embeddings)


Generating embeddings for 93 texts...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches: 100%|██████████| 3/3 [00:02<00:00,  1.35it/s]


Generated embeddings with shape: (93, 384)
Adding 93 documents to vector store...
Successfully added 93 documents to vector store
Total documents in collection: 93
